## Travailler avec un Notebook dans le village

J'expérimente ici pour voir comment il est possible de travailler avec une base qui ne contient que notre "village", faire des requêtes dedans mais sans toucher au contenu de la base.

In [6]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [7]:
from batid.models import Building

In [8]:
from contextlib import ContextDecorator
from django.db import transaction
from django.db import connection
from batid.tests.test_search import create_village

# création d'un décorateur qui vide la table building (et les tables associées en cascade)
class insert_village(ContextDecorator):
    def __enter__(self):
        cursor = connection.cursor()
        cursor.execute('BEGIN TRANSACTION')
        cursor.execute('TRUNCATE TABLE batid_building CASCADE')
        create_village()
        return self

    def __exit__(self, *exc):
        cursor.execute('ROLLBACK TRANSACTION')
        return False

# le décorateur précédent est appelé par le décorateur transaction.atomic() qui permet de ne pas faire de commit 
# dans la base après chaque execution SQL, mais de tout wrapper dans une transaction que l'on peut rollback 
# une fois terminée
def in_village(func):
    from django.db import transaction
    return transaction.atomic(insert_village()(func))

rajout d'une fonction de display de carte à titre d'exemple

In [9]:
import json
from ipyleaflet import Map, Marker, GeoJSON

def show_building(building):
    shape = building.shape
    shape_4326 = shape.transform(4326)
    geojson = json.loads(shape.geojson)
    building.point.transform(4326)

    [lon, lat] = building.point.coords
    m = Map(center=[lat, lon], zoom=17)
    geojson = GeoJSON(data=geojson)
    m.add_layer(geojson)
    display(m)

exemple de création d'une fonction que l'on décore avec @in_village, ce qui la place automatiquement dans un contexte où la base ne contient que le village

In [10]:
from django.db import transaction

@in_village
def test():
    building = Building.objects.all()[0]
    return building

In [14]:
print(f"la base contient {len(Building.objects.all())} batiments")
print("je fais une requête dans le village")
building = test()
show_building(building)
print(f"la base contient à la fin de l'opération {len(Building.objects.all())} batiments, comme initialement")

la base contient 0 batiments
je fais une requête dans le village


Map(center=[45.12729386087943, 5.5252574166289214], controls=(ZoomControl(options=['position', 'zoom_in_text',…

la base contient à la fin de l'opération 0 batiments
